# Pause and resume cloud scheduler jobs based on day period rain prediction

## Testing Cloud Scheduler methods

In [442]:
from google.cloud import scheduler_v1

class Scheduler:
    
    def __init__(self, sa_json=None):
        if sa_json is not None:
            self.client = scheduler_v1.CloudSchedulerClient.from_service_account_json(sa_json)
        else: self.client = scheduler_v1.CloudSchedulerClient()

    def pause_job(self, job_id):
        return self.client.pause_job(name=job_id)
        
    def resume_job(self, job_id):
        return self.client.resume_job(name=job_id)

### Custom scheduler instance

In [443]:
sa_json = '../../../../Apps/APIs/cams-rio/auth/octacity-iduff.json'

sched = Scheduler(sa_json)

### List jobs

In [445]:
parent = f'projects/{project}/locations/{location}'

jobs = sched.client.list_jobs(parent=parent)

jobs_names = [res.name for res in jobs]; jobs_names

['projects/octacity/locations/us-central1/jobs/alertario-job-switch',
 'projects/octacity/locations/us-central1/jobs/post-cameras-alertario',
 'projects/octacity/locations/us-central1/jobs/post-city',
 'projects/octacity/locations/us-central1/jobs/post-ipp-polygons-overview',
 'projects/octacity/locations/us-central1/jobs/post-polygons',
 'projects/octacity/locations/us-central1/jobs/post-polygons-alertario',
 'projects/octacity/locations/us-central1/jobs/post-polygons-ipp',
 'projects/octacity/locations/us-central1/jobs/post-polygons-overview',
 'projects/octacity/locations/us-central1/jobs/post-predict',
 'projects/octacity/locations/us-central1/jobs/record-comando',
 'projects/octacity/locations/us-central1/jobs/record-normal',
 'projects/octacity/locations/us-central1/jobs/record-polygons',
 'projects/octacity/locations/us-central1/jobs/record-rain',
 'projects/octacity/locations/us-central1/jobs/record-rivers-manual',
 'projects/octacity/locations/us-central1/jobs/record-waze']

### Pause Job manually

In [215]:
res = sched.pause_job('projects/octacity/locations/us-central1/jobs/record-rivers-manual')

# print(res)

### Resume Job manually

In [219]:
res = sched.resume_job('projects/octacity/locations/us-central1/jobs/record-rivers-manual')

# print(res)

---
# Scraping rain forecasat for next 24 hours from alertario website (Example)

#### Fetch updated data

In [507]:
import requests, pandas as pd
from lxml import etree
from datetime import datetime as dt 
# from modules.cloud_scheduler import Scheduler

class AlertaRio:
    
    baseURL = 'http://alertario.rio.rj.gov.br'
    rain_prediction_url = f'{baseURL}/upload/Previsao.html'
    
    day_periods = {
        'Manhã': '06:00:00',
        'Tarde': '12:00:00',
        'Noite': '18:00:00',
        'Madrugada': '00:00:00',
    }
    precipitation_categories = [
        'Sem chuva', 'Pancadas de chuva isoladas',
        'Chuva fraca a moderada isolada'
    ]

    msg_fail = msg = 'ALERTA-RIO JOB SWITCH SKIPPED · CURRENT DAY PERIOD UNAVAILABLE'
    msg_scss = '* ALERTA-RIO FORECAST JOB SWITCH · {} JOBS: {}'
    msg_error = 'ALERTA-RIO JOB SWITCH EXCEPTION CAUGHT · {}'

    def last_update(res=None):
        if res is None: res = requests.get(AlertaRio.rain_prediction_url, timeout=None) # get url res
        table = etree.HTML(res.text).find("body/p") # get html p element
        info = table.text.split(' ')  # list elemement text
        date, time = info[-1], info[-3] # get last update date and time
        return dt.strptime(f'{date} {time}', '%d/%m/%Y %H:%M') # parse date and time

    def next24hours(as_datetime=True, last_update=False): # Rain forecast web scraping
        df = pd.read_html(AlertaRio.rain_prediction_url)[0] # get html table
        df.columns = ['id'] + [df.columns[i][1] for i in range(1, 5)]
        df = df.iloc[1:5].set_index('id', drop=True).T
        df.columns = ['_'.join(col.lower().split(' ')) for col in df.columns]
        df[['período', 'data']] = [s.split('  ') for s in df.index]
        df['hora'] = df['período'].map(AlertaRio.day_periods)
        df['início'] = pd.to_datetime(df['data'] + ' ' + df['hora'], format='%d/%m/%Y %X')
#         df['início'] = (df['data'] + ' ' + df['hora']).astype('datetime64[s]')
        df['fim'] = df['início'] + pd.Timedelta(6, 'h')
        # ADD CURRENT AND NEXT DAY PERIOD FLAGS FIELDS
        if last_update: df['última_atualização'] = AlertaRio.last_update(res=None) 
        if not as_datetime:
            for col in ['início', 'fim'] + ['última_atualização'] if last_update else []:
                df[col] = df[col].dt.strftime('%Y/%m/%d %X')
        return df
    
    def rain_next():
        now = dt.now()
        df = AlertaRio.next24hours()
        df = df[df['início'] > now] # filter out predictions berore current day period
        if len(df): # first period after current day period
            return df.iloc[0]['precipitação'] != 'Sem chuva'
        return None

    def rain_now():
        now = dt.now()
        df = AlertaRio.next24hours()
        df = df[df['início'] < now] # filter out predictions after current day period
        if len(df): # last period before next day period
            return df.iloc[-1]['precipitação'] != 'Sem chuva'
        return None

    def __init__(self, sa_json):
        self.sched = Scheduler(sa_json)
        
    def job_switch(self, job_names, project, location):
        job_path = f'projects/{project}/locations/{location}/jobs'
        job_ids = [f'{job_path}/{job_name}' for job_name in job_names]
        try:
            rain_now = AlertaRio.rain_now()
            if rain_now is None:
                msg = AlertaRio.msg_fail
                print(msg); return msg
            switch = self.sched.resume_job if rain_now else self.sched.pause_job
            list(map(switch, job_ids))
            state = 'RESUMING' if rain_now else 'PAUSING'
            msg = AlertaRio.msg_scss.format(state, job_names)
            print(msg); return 'ALERTA-RIO JOB SWITCH SUCCESS'
        except Exception as e:
            msg = AlertaRio.msg_error.format(e)
            print(msg); return msg

#### Testing class methods

In [508]:
print('\n* LAST UPDATE: ', AlertaRio.last_update())
print('\n* RAIN ON NEXT DAY PERIOD: ', AlertaRio.rain_next())
print('\n* RAIN ON THIS DAY PERIOD: ', AlertaRio.rain_now())


* LAST UPDATE:  2023-04-18 16:16:00

* RAIN ON NEXT DAY PERIOD:  True

* RAIN ON THIS DAY PERIOD:  True


In [509]:
df = AlertaRio.next24hours(as_datetime=False); df

,céu,precipitação,vento,tendência_da_temperatura,período,data,hora,início,fim
Noite 18/04/2023,Nublado,Pancadas de chuva isoladas,Moderado a Forte de S/SW,Estável,Noite,18/04/2023,18:00:00,2023-04-18 18:00:00,2023-04-19 00:00:00
Madrugada 19/04/2023,Nublado a Encoberto,Pancadas de chuva isoladas,Moderado de S/SW,Estável,Madrugada,19/04/2023,00:00:00,2023-04-19 00:00:00,2023-04-19 06:00:00
Manhã 19/04/2023,Nublado a Encoberto,Chuva fraca a moderada isolada,Moderado a Forte de S/SW,Declínio,Manhã,19/04/2023,06:00:00,2023-04-19 06:00:00,2023-04-19 12:00:00
Tarde 19/04/2023,Nublado a Encoberto,Chuva fraca a moderada isolada,Moderado de S/SW,Declínio,Tarde,19/04/2023,12:00:00,2023-04-19 12:00:00,2023-04-19 18:00:00


---
## Pause or resume jobs conditionally

### Pause or resume jobs based on rain forecast for current period

#### Basic example

In [511]:
if df.iloc[0]['precipitação'] == 'Sem chuva':
    print('Sem chuva para esse período')
else:
    print('Com chuva para esse período')

Com chuva para esse período


#### Pause/resume single job

In [513]:
project = 'octacity'
location = 'us-central1'
job_name = 'record-rivers-manual'

parent = f'projects/{project}/locations/{location}'
job_path = f'{parent}/jobs'
job_id = f'{job_path}/{job_name}'

# Custom scheduler instance
sa_json = '../../../../Apps/APIs/cams-rio/auth/octacity-iduff.json'

alertario = AlertaRio(sa_json)

# Rain forecast web scraping
df = AlertaRio.next24hours()
period = df.iloc[0]

if not AlertaRio.rain_now():
    print(f'\n* Sem chuva para esse período ({period["período"]} · {period["data"]})')
    print(f'\n- Pausando Job: {job_name}')
    res = alertario.sched.pause_job(job_id)
else:
    print(f'\n* Com chuva para esse período ({period["período"]} · {period["data"]})')
    print(f'\n- Resumingo Job: {job_name}')
    res = alertario.sched.resume_job(job_id)


* Com chuva para esse período (Noite · 18/04/2023)

- Resumingo Job: record-rivers-manual


#### Pause/resume list of jobs

In [515]:
# Parameter configuration

project = 'octacity'
location = 'us-central1'

job_names = [
    'post-cameras-alertario',
    'post-city',
    'post-ipp-polygons-overview',
    'post-polygons',
    'post-polygons-alertario',
    'post-polygons-ipp',
    'post-polygons-overview',
    'post-predict',
    'record-comando',
    'record-polygons',
    'record-rivers-manual',
    'record-waze'
]

sa_json = '../../../../Apps/APIs/cams-rio/auth/octacity-iduff.json'
alertario = AlertaRio(sa_json)

# Weather prediction based job switching

alertario.job_switch(job_names, project, location)

* ALERTA-RIO FORECAST JOB SWITCH · RESUMING JOBS: ['post-cameras-alertario', 'post-city', 'post-ipp-polygons-overview', 'post-polygons', 'post-polygons-alertario', 'post-polygons-ipp', 'post-polygons-overview', 'post-predict', 'record-comando', 'record-polygons', 'record-rivers-manual', 'record-waze']


'ALERTA-RIO JOB SWITCH SUCCESS'